In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from datetime import datetime

# Import the data
data = pd.read_csv('/Users/walterlopez/ASTR-19_2024/example/ASTR19_F23_group_project_data.txt', delim_whitespace=True, comment='#', header=None)
data.columns = ['Day', 'Time', 'TideHeight']
# Convert the time to decimal hours
def time_to_decimal(t):
    h, m = map(int, t.split(':'))
    return h + m / 60

data['DecimalTime'] = data['Time'].apply(time_to_decimal)
data['TimeInDays'] = data['Day'] + data['DecimalTime'] / 24

# Define the oscillatory function
def tide_height(time, amplitude_1, period_1, amplitude_2, period_2):
    return amplitude_1 * np.sin(2 * np.pi * time / period_1) + amplitude_2 * np.sin(2 * np.pi * time / period_2)

# Fit the function to the data
initial_guess = [1, 24, 0.5, 12.42] # Initial guess for the amplitudes and periods
params, covariance = curve_fit(tide_height, data['TimeInDays'], data['TideHeight'], p0=initial_guess)
fitted_tide_heights = tide_height(data['TimeInDays'], *params)

# Plot the fitted curve and the data
plt.figure(figsize=(10, 5))
plt.plot(data['TimeInDays'], data['TideHeight'], 'bo', label='Data')
plt.plot(data['TimeInDays'], fitted_tide_heights, 'r-', label='Fit')
plt.xlabel('Time (Days)')
plt.ylabel('Tide Height (ft)')
plt.title('Tidal Measurements and Oscillatory Model Fit')
plt.legend()
plt.grid(True)
plt.savefig('tidal_fit.pdf')
plt.close()

# Calculate residuals
residuals = data['TideHeight'] - fitted_tide_heights

# Plot a histogram of the residuals
plt.figure(figsize=(10, 5))
plt.hist(residuals, bins='auto', alpha=0.7, label='Residuals')
plt.xlabel('Residuals (ft)')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals')
plt.legend()
plt.grid(True)
plt.savefig('residuals_histogram.pdf')
plt.close()

# Calculate the standard deviation of the residuals
std_dev = np.std(residuals)
print(f"The standard deviation of the residuals is: {std_dev}")

# Adding an outlier due to the tsunami
outlier_value = 2  # Tsunami added 2ft to the high tide
residuals_with_outlier = np.append(residuals, outlier_value - np.mean(residuals))

# Plot a histogram with the outlier
plt.figure(figsize=(10, 5))
plt.hist(residuals_with_outlier, bins='auto', alpha=0.7, label='Residuals with Outlier')
plt.xlabel('Residuals (ft)')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals with Outlier')
plt.legend()
plt.grid(True)
plt.savefig('residuals_with_outlier_histogram.pdf')
plt.close()

# Calculate how many standard deviations the outlier is from the mean
tsunami_deviation = (outlier_value - np.mean(residuals)) / std_dev
print(f"The tsunami outlier is {tsunami_deviation:.2f} standard deviations away from the mean of the residuals.")


The standard deviation of the residuals is: 2.4742085720331235
The tsunami outlier is -0.09 standard deviations away from the mean of the residuals.
